# Laboratorio 7
## 1) Image deblur
Il problema di deblur consiste nella ricostruzione di un immagine a partire da un dato acquisito mediante il seguente modello:
$$ y = Ax + \eta $$
dove :
• $y$ rappresenta l’immagine corrotta,
• $x$ rappresenta l’immagine originale che vogliamo ricostruire
• $A$ rappresenta l’operatore che applica il blur Gaussiano
• $\eta \sim \mathcal{N}(0, \sigma^2)$ rappresenta una realizzazione di rumore additivo con distribuzione Gaussiana di media $\mu = 0$ e deviazione standard $\sigma$.

### Esercizio 1:
• Caricare l’immagine camera dal modulo `skimage.data` rinormalizzandola nel range $[0, 1]$.
• Applicare un blur di tipo gaussiano con deviazione standard 3 il cui kernel ha dimensioni 24 × 24. Utilizzare prima `cv2` (`open-cv`) e poi la trasformata di Fourier.
• Aggiungere rumore di tipo gaussiano, con $\sigma = 0.02$, usando la funzione `np.random.normal()`.
• Calcolare le metriche Peak Signal Noise Ratio (PSNR) e Mean Squared Error (MSE) tra l’immagine degradata e l’immagine esatta usando le funzioni `peak_signal_noise_ratio` e `mean_squared_error` disponibili nel modulo `skimage.metrics`.

In [6]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, metrics
from scipy import signal
from numpy import fft
import sys
from lib.utils import psf_fft, A, AT, gaussian_kernel

# Immagine in floating point con valori tra 0 e 1
X = ...
m, n = X.shape

# Genera il filtro di blur
k = gaussian_kernel(...)
plt.imshow(k)
plt.show()

# Blur with openCV
X_blurred = cv.filter2D(...)
plt.subplot(121).imshow(X, cmap='gray', vmin=0, vmax=1)
plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122).imshow(X_blurred, cmap='gray', vmin=0, vmax=1)
plt.title('Blurred')
plt.xticks([]), plt.yticks([])
plt.show()

# Blur with FFT
K = psf_fft(...)
plt.imshow(np.abs(K))
plt.show()

X_blurred = ...

# Genera il rumore
sigma = 0.02
np.random.seed(42)
noise = np.random.normal(size=X.shape) ...

# Aggiungi blur e rumore
y = ...
PSNR = metrics.peak_signal_noise_ratio(...)
ATy = AT(y, K)


# Visualizziamo i risultati
plt.figure(figsize=(30, 10))
plt.subplot(121).imshow(X, cmap='gray', vmin=0, vmax=1)
plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122).imshow(y, cmap='gray', vmin=0, vmax=1)
plt.title(f'Corrupted (PSNR: {PSNR:.2f})')
plt.xticks([]), plt.yticks([])
plt.show()

SyntaxError: invalid syntax (3915169647.py, line 39)

### Esercizio 2: soluzione naive
Una possibile ricostruzione dell’immagine originale $x$ partendo dall’immagine corrotta $y$ è la soluzione naive data dal minimo del seguente problema di ottimizzazione:
$$ x^\ast = \arg \min_{x} \dfrac{1}{2} {\|Ax − y\|}^2_2 $$
• Utilizzando il metodo del gradiente coniugato implementato dalla funzione `minimize` della libreria `scipy`, calcolare la soluzione naive.
• Analizza l’andamento del PSNR e dell’MSE al variare del numero di iterazioni.

In [7]:
from scipy.optimize import minimize

# Funzione da minimizzare
def f(x):
    x = x.reshape((m, n))
    Ax = A(x, K)
    return 0.5 * np.sum(np.square(Ax - y))

# Gradiente della funzione da minimizzare
def df(x):
    x = x.reshape((m, n))
    ATAx = AT(A(x,K),K)
    d = ATAx - ATy
    return d.reshape(m * n)

# Minimizzazione della funzione
x0 = y.reshape(m*n)
max_iter = 25
res = minimize(..., method='CG', jac=df, options={'maxiter':max_iter, 'return_all':True})

# Per ogni iterazione calcola il PSNR rispetto all'originale
PSNR = np.zeros(max_iter + 1)
for k, x_k in enumerate(res.allvecs):
    PSNR[k] = metrics.peak_signal_noise_ratio(X, x_k.reshape(X.shape))

# Risultato della minimizzazione
X_res = res.x.reshape((m, n))

# PSNR dell'immagine corrotta rispetto all'oginale
starting_PSNR = np.full(PSNR.shape[0], metrics.peak_signal_noise_ratio(X, y))

# Visualizziamo i risultati
ax2 = plt.subplot(1, 2, 1)
ax2.plot(PSNR, label="Soluzione naive")
ax2.plot(starting_PSNR, label="Immagine corrotta")
plt.legend()
plt.title('PSNR per iterazione')
plt.ylabel("PSNR")
plt.xlabel('itr')
plt.subplot(1, 2,2).imshow(X_res, cmap='gray', vmin=0, vmax=1)
plt.title('Immagine Ricostruita')
plt.xticks([]), plt.yticks([])
plt.show()

NameError: name 'y' is not defined

### Esercizio 3: soluzione regolarizzata
Si consideri il seguente problema regolarizzato secondo Tikhonov:
$$ x^\ast = \arg \min_{x} \dfrac{1}{2} {\|Ax − y\|}^2_2 + \lambda {\|x\|}^2_2 $$
• Utilizzando sia il metodo del gradiente che il metodo del gradiente coniugato calcolare la soluzione del problema regolarizzato.
• Analizzare l’andamento del PSNR e dell’MSE al variare del numero di iterazioni.
• Facendo variare il parametro di regolarizzazione $\lambda$, analizzare come questo influenza le prestazioni del metodo analizzando le immagini.
• Scegliere $\lambda$ con il metodo di discrepanza.
• Scegliere $\lambda$ attraverso test sperimentali come il valore che massimizza il valore del PSNR. Confrontare il valore ottenuto con quella della massima discrepanza.

In [8]:
# Funzione da minimizzare
def f(x, L):
    nsq = np.sum(np.square(x))
    x  = x.reshape((m, n))
    Ax = A(x, K)
    return 0.5 * np.sum(np.square(Ax - y)) + 0.5 * L * nsq

# Gradiente della funzione da minimizzare
def df(x, L):
    Lx = L * x
    x = x.reshape(m, n)
    ATAx = AT(A(x,K),K)
    d = ATAx - ATy
    return d.reshape(m * n) + Lx

x0 = y.reshape(m*n)
lambdas = [0.01,0.03,0.04, 0.06]
PSNRs = []
images = []

# Ricostruzione per diversi valori del parametro di regolarizzazione
for i, L in enumerate(lambdas):
    # Esegui la minimizzazione con al massimo 50 iterazioni
    max_iter = 50
    res = minimize(f, x0, (L), method='CG', jac=df, options={'maxiter':max_iter})

    # Aggiungi la ricostruzione nella lista images
    X_curr = res.x.reshape(X.shape)
    images.append(X_curr)

    # Stampa il PSNR per il valore di lambda attuale
    PSNR = metrics.peak_signal_noise_ratio(X, X_curr)
    PSNRs.append(PSNR)
    print(f'PSNR: {PSNR:.2f} (\u03BB = {L:.2f})')
    
    

# Visualizziamo i risultati
plt.plot(lambdas,PSNRs)
plt.title('PSNR per $\lambda$')
plt.ylabel("PSNR")
plt.xlabel('$\lambda$')
plt.show()

plt.figure(figsize=(30, 10))

plt.subplot(1, len(lambdas) + 2, 1).imshow(X, cmap='gray', vmin=0, vmax=1)
plt.title("Originale")
plt.xticks([]), plt.yticks([])
plt.subplot(1, len(lambdas) + 2, 2).imshow(y, cmap='gray', vmin=0, vmax=1)
plt.title("Corrotta")
plt.xticks([]), plt.yticks([])


for i, L in enumerate(lambdas):
  plt.subplot(1, len(lambdas) + 2, i + 3).imshow(images[i], cmap='gray', vmin=0, vmax=1)
  plt.title(f"Ricostruzione ($\lambda$ = {L:.2f})")
plt.show()

NameError: name 'y' is not defined

### Esercizio 4: downsapling
• Ripetere i punti precedenti utilizzando anche l’operatore downsampling con i seguenti fattori di scaling $sf = 2, 4, 8, 16$.
• Testare i punti precedenti su due immagini in scala di grigio con caratteristiche differenti (per esempio, un’immagine di tipo fotografico e una ottenuta con uno strumento differente, microscopio o altro).
• Degradare le nuove immagini applicando, mediante le funzioni gaussian_kernel(), psf_fft(), l’operatore di blur con parametri:
– $\sigma = 0,5$ dimensione del kernel 7 × 7 e 9 × 9
– $\sigma = 1,3$ dimensione del kernel 5 × 5
– Aggiungendo rumore gaussiano con deviazione standard nell’ intervallo $(0, 0, 05]$.

### Esercizio 5: Variazione Totale (facoltativo)
Un’altra funzione adatta come termine di regolarizzazione è la Variazione Totale. Data $x$ immagine di dimensioni m × n, la variazione totale $T V$ di $x$ è definita come:
$$ T V(x) = \sum_{i}^{m} \sum_{j}^{m} \sqrt{\| \nabla x(i,j)\|^2_2 + \epsilon^2} $$
Come nei casi precedenti il problema di minimo che si va a risolvere è il seguente:
$$ x^\ast = \arg \min_{x} \dfrac{1}{2} {\|Ax − b\|}^2_2 + \lambda T V(x) $$
il cui gradiente $\nabla f$ è dato da
$$ \nabla f(x) = (A^T Ax - A^T b) + \lambda \nabla T V(x) $$
• Utilizzando il metodo del gradiente e la funzione `minimize`, calcolare la soluzione del precendente problema di minimo regolarizzato con la funzione TV per differenti valori di λ, utilizzando le funzioni `totvar` e `grad_totvar`.
• Per calcolare il gradiente dell’immagine $\nabla u$ usiamo la funzione `np.gradient` che approssima la derivata per ogni pixel calcolando la differenza tra pixel adiacenti. I risultati sono due immagini della stessa dimensione dell’immagine in input, una che rappresenta il valore della derivata orizzontale e l’altra della derivata verticale . Il gradiente dell’immagine nel punto $(i, j)$ è quindi un vettore di due componenti, uno orizzontale contenuto e uno verticale.
• Per risolvere il problema di minimo è necessario anche calcolare il gradiente della variazione totale che è definito nel modo seguente:
$$ \nabla T V(u) = -div\bigg(\dfrac{\nabla u}{\sqrt{\| \nabla u \|^2_2 + \epsilon^2}}\bigg) $$
dove la divergenza è definita come: 
$$ div(F) = \dfrac{\partial F_x}{\partial x} + \dfrac{\partial F_y}{\partial y} $$
$div(F)$ è la divergenza del campo vettoriale F, nel nostro caso F ha due componenti dati dal gradiente dell’immagine $\nabla u$ scalato per il valore $\dfrac{1}{\sqrt{\| \nabla u \|^2_2 + \epsilon^2}}$. Per calcolare la divergenza bisogna calcolare la derivata orizzontale $\dfrac{\partial F_x}{\partial x}$ della componente $x$ di F e sommarla alla derivata verticale $\dfrac{\partial F_y}{\partial y}$ della componente $y$ di F . Per specificare in quale direzione calcolare la derivata con la funzione `np.gradient` utilizziamo il parametro `axis = 0` per l’orizzontale e `axis = 1` per la verticale.